In [1]:
import numpy as np
import pandas as pd

from src.constant import DATA_DIR, DATABASE_DIR, MAIN_DIR
from src.database import DB
from src.database.queries import *
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

In [2]:
# MAX_COST = 8.9
# MAX_TIME = 0.89
# MAX_COST = 4.7
# MAX_TIME = 0.47
# DIR = DATABASE_DIR
# DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-03-05" / "10" / "200" / "database")
# DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-03-05" / "531" / "200" / "database")

# MAX_COST = 41.0
# MAX_TIME = 4.1
# MAX_COST = 21.8
# MAX_TIME = 2.18
MAX_COST = 10.9
MAX_TIME = 1.09
# DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-03-05" / "10" / "400" / "database")
# DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-03-05" / "531" / "400" / "database")
DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-03-05" / "266" / "400" / "database")

# MAX_COST = 100.0
# MAX_TIME = 10.0
# MAX_COST = 53.1
# MAX_TIME = 5.31
# DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "600" / "database")
# DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-03-05" / "531" / "600" / "database")

generators = [
    "cluster_netgen",
    "compression",
    "expansion",
    "explosion",
    "grid",
    "cluster",
    "implosion",
    "linearprojection",
    "rotation",
    "uniform_portgen",
]

test_instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TEST_600" / "index.json")

id_to_generator_name = {
    instance.id(): instance.filepath.parts[-2] for instance in test_instances
}

id_to_name = {
    instance.id(): instance.filepath.parts[-1] for instance in test_instances
}

In [14]:
def agg_prefix(prefix):

    frames = []

    def agg_cost(x):
        count = x.shape[0]
        x = x[x < 26.6]
        x = x.sort_values()
        idx = count // 2
        if x.shape[0] <= idx:
            return x.iloc[-1]
        return x.iloc[idx]
        

    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        print(db_path)
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        series = (
            results.loc[results["prefix"].str.startswith("test")]
            .groupby(["instance_id", "prefix"])["cost"]
            .min()
            .reset_index()
            .groupby("instance_id")["cost"]
            .agg(agg_cost)
        )
        frames.append(series)
        

    df = (
        pd.concat(frames, axis=1)
        .mean(axis=1)
        .round(3)
        .rename(prefix)
        .to_frame()
    )

    df["generator"] = df.index.map(id_to_generator_name)
    df["name"] = df.index.map(id_to_name)
    df = df.pivot_table(index="generator", columns="name", values=prefix).loc[generators, :]
    return df

n = 250
plain = agg_prefix(f"run-plain-{n}")
plain = plain.assign(generator=f"plain-{n}").set_index("generator", append=True)

C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983381.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983382.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983383.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983384.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983385.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983386.db
C:\Users\zakrz\Documents\DataScience\praca magisters

In [ ]:
def agg_cost(x):
    count = x.shape[0]
    x = x[x < 26.6]
    x = x.sort_values()
    idx = count // 2
    if x.shape[0] <= idx:
        return x.iloc[-1]
    return x.iloc[idx]

db_path = DIR / f"run-plain-30-981927.db"
db = DB(db_path)
results = pd.read_sql_query("SELECT * FROM results", db._conn)
series = (
    results.loc[results["prefix"].str.startswith("test")]
    .groupby(["instance_id", "prefix"])["cost"]
    .min()
    .reset_index()
    .groupby("instance_id")["cost"]
    .agg(agg_cost)
)
series

# frame = series.to_frame()
# frame["generator"] = frame.index.map(id_to_generator_name)
# frame["name"] = frame.index.map(id_to_name)
# frame = frame.pivot_table(index="generator", columns="name", values="cost").loc[generators, :]
# print(frame.mean().mean())
# # frame

# # 980931 --> 2.1948
# # 980935 --> 5.2959
# # 980937 --> 3.8926

In [15]:
print(f"mean: {plain.mean().mean():.3f}")

mean: 0.728


In [16]:
df = plain.copy().sort_index(level=0).loc[generators]   
df["mean"] = df.mean(axis=1)

In [17]:
df.style.background_gradient(cmap="Reds", vmin=0, vmax=5).format("{:.3f}")

,name,000.tsp,001.tsp,002.tsp,003.tsp,004.tsp,mean
generator,generator,,,,,,
cluster_netgen,plain-250,0.730,1.157,1.316,1.124,0.518,0.969
compression,plain-250,0.370,0.477,0.640,0.563,0.652,0.540
expansion,plain-250,0.695,1.126,0.511,0.389,0.824,0.709
explosion,plain-250,0.278,0.353,1.277,0.646,0.684,0.648
grid,plain-250,0.184,0.537,1.226,0.278,0.560,0.557
cluster,plain-250,0.529,0.607,0.624,0.717,1.457,0.787
implosion,plain-250,0.842,0.437,0.318,1.257,2.457,1.062
linearprojection,plain-250,0.341,0.365,0.540,0.275,0.950,0.494
rotation,plain-250,1.329,0.874,1.070,0.761,0.514,0.910


In [18]:
df.to_excel("tmp.xlsx")

In [19]:
def agg_prefix(prefix):
    times = {}
    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        times[idx] = results.loc[
            results["prefix"].str.startswith("config")
            & results["cached"].eq(0)
            & results["surrogate"].eq(0),
            "time",
        ].sum() / 3600
    return np.round(pd.Series(times).mean(), 2)

plain = agg_prefix(f"run-plain-{n}")
print(f"plain: {plain:.2f} h")

plain: 9.63 h


In [20]:
# skip cutt-offs
def agg_prefix(prefix):
    times = {}
    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        db = DB(db_path)
        print(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        times[idx] = results.loc[
            results["prefix"].str.startswith("config")
            & results["cached"].eq(0)
            & results["surrogate"].eq(0)
            & results["cost"].lt(MAX_COST),
            "time",
        ].mean()
    return np.round(pd.Series(times).mean(), 2)

plain = agg_prefix(f"run-plain-{n}")
print(f"plain: {plain:.2f}")

C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983381.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983382.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983383.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983384.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983385.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\_archive\experiments\2025-03-05\266\400\database\run-plain-250-983386.db
C:\Users\zakrz\Documents\DataScience\praca magisters

In [13]:
# % cutt-offs
def agg_prefix(prefix):
    times = {}
    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        times[idx] = results.loc[
            results["prefix"].str.startswith("config")
            & results["cached"].eq(0)
            & results["surrogate"].eq(0),
            "cost"
        ].eq(MAX_COST).mean()
    return np.round(pd.Series(times).mean(), 2)

plain = agg_prefix(f"run-plain-{n}")
print(f"plain: {plain:.2f}")

plain: 0.76
